In [7]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot
from numpy import linalg as LA
import math

def create_vector(a,b,step):
    new_vec = []
    curr_val = a
    while curr_val < b:
        new_vec.append(curr_val)
        curr_val += step
        
    return new_vec

df_dx = lambda f,x : diff(f,x)

D = lambda Q, x : vector([df_dx(f, x) for f in Q])

DD = lambda Q, x, y : vector([df_dx(df_dx(f, x),y) for f in Q])

def cross_product(v_1, v_2):
    
    return vector([v_1[1]*v_2[2] - v_1[2]*v_2[1], - v_1[0]*v_2[2] + v_1[2]*v_2[0], v_1[0]*v_2[1] - v_1[1]*v_2[0]])

def draw_alpha_complex(alpha_complex, S, a1):
    
    fig = go.Figure()
    
    things_to_plot = []
    
    # Set axes properties
    fig.update_xaxes(range=[-5, 5], zeroline=False)
    fig.update_yaxes(range=[-2, 8])

    filtered_nodes = []
    
    for xyz, simplex in zip(S, alpha_complex[0]):
        
        if simplex[1] < a1: 
            filtered_nodes.append(xyz)
            
    filtered_nodes = np.array(filtered_nodes)
        
    things_to_plot.append(go.Scatter3d(x=filtered_nodes[:,0], y=filtered_nodes[:,1], z=filtered_nodes[:,2],
            mode='markers',
            marker=dict(
                size=12,
                color=filtered_nodes[:,2],                # set color to an array/list of desired values
                colorscale='Viridis',   # choose a colorscale
                opacity=0.8
            ), showlegend=False))
    
    for simplex, weight in alpha_complex[1]:
        if weight < a1:
            things_to_plot.append(go.Scatter3d(x=[S[simplex[0]][0], S[simplex[1]][0]], y=[S[simplex[0]][1], S[simplex[1]][1]], z=[S[simplex[0]][2], S[simplex[1]][2]], mode='lines',line=dict(
                                                color="black",
                                                width=10),showlegend=False))
        
    for simplex, weight in alpha_complex[2]:
        if weight < a1:
            X = [S[simplex[0]][0], S[simplex[1]][0], S[simplex[2]][0]]
            Y = [S[simplex[0]][1], S[simplex[1]][1], S[simplex[2]][1]]
            Z = [S[simplex[0]][2], S[simplex[1]][2], S[simplex[2]][2]]
            
            i = np.array([0])
            j = np.array([1])
            k = np.array([2])
            
            things_to_plot.append(go.Mesh3d(x=X, y=Y, z=Z,alphahull=5, opacity=0.4, color='cyan', i=i, j=j, k=k))
        
    layout = go.Layout(
        autosize=False,
        width=1000,
        height=1000
    )
        
    fig = go.Figure(data=things_to_plot, layout=layout)
        
    fig.show()

In [8]:
#S = np.array([[0,0,0], [1,0,0], [4,0,0], [0,3,0]]) # flat

S = np.array([[0,0,0], [0,1,0], [1,1,0], [1,0,0]])


# S = np.array([[0,0,0], [0,1,0], [0,0,1], [1,1,1]])
# S = np.array([[0,0,0], [0,1,0], [0,0,1], [1,0,1]])

In [9]:
my_alpha_complex = {
    0: [([0],0), ([1],0), ([2],0), ([3],0)],
    1: [([0,1],0), ([1,2],0), ([1,3],0), ([0,3],0), ([2,3],0)],
    2: [([0,1,3],0), ([1,2,3],0)]
}

In [10]:
draw_alpha_complex(my_alpha_complex, S, 1)

In [13]:
def wald_curvature(alpha_complex, S, edge):
    
    p2_index = edge[0]
    p4_index = edge[1]
    
    connecting_indices = []
    for simplex, weight in alpha_complex[2]: 
        if p2_index in simplex and p4_index in simplex: 
            connecting_indices.append([e for e in simplex if ( e != p2_index and e != p4_index)][0])
            
    if len(connecting_indices) != 2:
        print("Either boundary or not of dimension 2")
        return 
    
    p1_index = connecting_indices[0]
    p3_index = connecting_indices[1]
    
    v1 = S[p1_index] - S[p2_index] 
    v2 = S[p4_index] - S[p2_index] 
    v3 = S[p3_index] - S[p2_index]
    
    angle_2 = math.acos(v1.dot(v2) / ( LA.norm(v1) * LA.norm(v2) ) )
    angle_2_prime = math.acos(v3.dot(v2) / ( LA.norm(v3) * LA.norm(v2) ) )
    
    d24 = LA.norm(v2)
    d12 = LA.norm(v1)
    d23 = LA.norm(v3)
    
    return 6 * (math.cos(angle_2) + math.cos(angle_2_prime)) / (d24 * (d12 * (math.sin(angle_2))^2 + d23 * (math.sin(angle_2_prime))^2))

In [14]:
p2 = 1
p4 = 3

wald_curvature(my_alpha_complex, S, [p2,p4])

TypeError: ufunc 'bitwise_xor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''